## 2. 뇌졸중 예측 모델 학습 (로지스틱 회귀) 
 * 데이터 로드 및 전처리
 * 모델 학습 (Logistic Regression)
 * 모델 평가 (ROC Curve, AUC Score)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve, classification_report
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt

### 1. 데이터 로드 및 전처리

In [ ]:
try:
    data = pd.read_csv('../data/health_checkup_data.csv')
    print("데이터 로드 성공!")
except FileNotFoundError:
    print("오류: '../data/health_checkup_data.csv' 파일을 찾을 수 없습니다.")

In [ ]:
if 'data' in locals():
    # PPT 기반 주요 변수 [cite: 70, 242]
    # (실제 프로젝트에 사용한 변수로 수정하세요)
    numeric_features = ['age', 'bmi']
    categorical_features = ['gender', 'hypertension', 'smoking_status']
    target = 'stroke'
    
    # 결측치 처리 (간단한 예시)
    data = data.dropna(subset=numeric_features + categorical_features + [target])

    X = data[numeric_features + categorical_features]
    y = data[target]
    
    # 데이터 분할
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    
    print(f"학습 데이터: {X_train.shape}, 테스트 데이터: {X_test.shape}")

### 2. 전처리 파이프라인 구성

In [ ]:
if 'data' in locals():
    # 숫자형: 결측치 중앙값 대체, 표준화
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])
    
    # 범주형: 결측치 'most_frequent' 대체, 원핫인코딩
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])
    
    # ColumnTransformer로 파이프라인 결합
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ])

### 3. 모델 학습 (로지스틱 회귀)

In [ ]:
if 'data' in locals():
    # 로지스틱 회귀 모델 파이프라인 생성 [cite: 68, 244]
    model_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', LogisticRegression(random_state=42))
    ])
    
    # 모델 학습
    model_pipeline.fit(X_train, y_train)
    
    print("모델 학습 완료.")

### 4. 모델 평가

In [ ]:
if 'data' in locals():
    # 1. 예측 (확률)
    y_train_proba = model_pipeline.predict_proba(X_train)[:, 1]
    y_test_proba = model_pipeline.predict_proba(X_test)[:, 1]
    
    # 2. AUC 스코어
    # PPT의 결과(Train 0.77, Test 0.80) [cite: 257, 275]와 유사하게 나오는지 확인
    train_auc = roc_auc_score(y_train, y_train_proba)
    test_auc = roc_auc_score(y_test, y_test_proba)
    
    print(f"Train AUC Score: {train_auc:.4f} (PPT: 0.77)")
    print(f"Test AUC Score: {test_auc:.4f} (PPT: 0.80)")

    # 3. 분류 리포트 (Test)
    y_test_pred = model_pipeline.predict(X_test)
    print("\nTest Set Classification Report:")
    print(classification_report(y_test, y_test_pred))

### 5. ROC 커브 시각화

In [ ]:
if 'data' in locals():
    fpr_train, tpr_train, _ = roc_curve(y_train, y_train_proba)
    fpr_test, tpr_test, _ = roc_curve(y_test, y_test_proba)
    
    plt.figure(figsize=(12, 6))
    
    # Train ROC Curve
    plt.subplot(1, 2, 1)
    plt.plot(fpr_train, tpr_train, label=f'Classifier (AUC = {train_auc:.2f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.title('Train ROC Curve - stroke [cite: 249]')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc='lower right')
    
    # Test ROC Curve
    plt.subplot(1, 2, 2)
    plt.plot(fpr_test, tpr_test, label=f'Classifier (AUC = {test_auc:.2f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.title('Test ROC Curve - stroke [cite: 273]')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc='lower right')
    
    plt.tight_layout()
    plt.show()